# Let's start with a cleaned up version of where we ended last time

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
#matplotlib inline

In [5]:
# read all the words
words = open('names.txt', 'r').read().splitlines()

# build stoi, itos
chars = sorted(list(set(''.join(words)))) # a to z
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

# build the training dataset
def build_dataset(words):
    block_size = 3
    X, Y = [], []
    
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

print(n1, n2-n1, len(words) - n2)
print(Xtr.shape, Xtr.dtype, Ytr.shape, Ytr.dtype)

25626 3203 3204
torch.Size([182625, 3]) torch.int64 torch.Size([182625]) torch.int64


In [6]:
# 30x200x27 neural net
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200,27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True
    
print(sum(p.nelement() for p in parameters))

11897


In [7]:
lossi, stepi = [], []

In [8]:
for i in range(200000):
    # minibatch construct with size = 32
    ix = torch.randint(0, Xtr.shape[0], (32,))
    
    # forward pass
    emb = C[Xtr[ix]] # 32x3x10
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2 # 32x27
    loss = F.cross_entropy(logits, Ytr[ix])
    
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    
    # track stats
    # we use log loss for tracking since log squashes the hockey-stick appearance of the previous graph, making it look nicer
    lossi.append(loss.log10().item())
    stepi.append(i)

In [9]:
def overall_loss(X, Y):
    emb = C[X]
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2
    return F.cross_entropy(logits, Y)    

print(overall_loss(Xtr, Ytr))
print(overall_loss(Xdev, Ydev))

tensor(2.1171, grad_fn=<NllLossBackward0>)
tensor(2.1622, grad_fn=<NllLossBackward0>)


In [ ]:
plt.plot(stepi, lossi)